### Data Merge

##### Declarations

In [1]:
#Defining libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#Importing helper functions
%run "helper_functions.ipynb"

In [3]:
#Input and output paths for all pickle files. To be used to load and save these files
#Output
input_path="C:\\Users\\Anuvrat\\OneDrive\\Documents\\milestone1\\data\\inputs"
output_path="C:\\Users\\Anuvrat\\OneDrive\\Documents\\milestone1\\data\\inputs"

In [75]:
#Loading input pickle files
df_income_1901_edited=pd.read_pickle(input_path+"\\df_income_1901_edited.pkl")
df_health_outcomes_edited=pd.read_pickle(input_path+"\\df_health_outcomes_edited.pkl")
df_sdoh_edited=pd.read_pickle(input_path+"\\df_sdoh_edited.pkl")
df_state_politics_edited=pd.read_pickle(input_path+"\\df_state_politics_edited.pkl")
df_state_public_spend_edited=pd.read_pickle(input_path+"\\df_state_public_spend_edited.pkl")
df_acs_edited=pd.read_pickle(input_path+"\\df_acs_edited.pkl")

In [36]:
#Printing shapes
for df in [df_income_1901_edited, df_health_outcomes_edited, df_sdoh_edited, df_state_politics_edited, df_state_public_spend_edited, df_acs_edited]:
    print(df.shape)

(33774, 15)
(1169992, 10)
(291024, 5)
(57, 3)
(515, 6)
(33120, 14)


In [71]:
df_income_1901_edited.ZIP.nunique(), df_health_outcomes_edited.ZIP.nunique(), df_sdoh_edited.ZIP.nunique(), df_acs_edited.ZIP.nunique()

(33774, 32409, 32336, 33120)

In [79]:
df_income_1901_essential=df_income_1901_edited[['ZIP', 'Estimate Households Total', 'Percent Households lt 10k', 
                                                'Estimate Households Median income (dollars)', 'Estimate Families Total',
                                                'Estimate Families Median income (dollars)', 'Estimate Nonfamily households Total',
                                                'Estimate Nonfamily households Median income (dollars)', 'Percent Households that are Families']]
df_income_1901_essential.shape

(33774, 9)

In [68]:
df_health_outcomes_essential=df_health_outcomes_edited[['ZIP', 'TotalPopulation', 'Short_Question_Text', 'Data_Value']]
df_health_outcomes_essential_wide=df_health_outcomes_essential.pivot(index=['ZIP', 'TotalPopulation'], columns='Short_Question_Text', 
                                                                     values='Data_Value').reset_index().rename_axis(None, axis=1)
#display(df_health_outcomes_essential_wide.head())
df_health_outcomes_essential_wide.shape

(32409, 39)

In [77]:
df_sdoh_essential_wide=df_sdoh_edited.pivot(index=['ZIP', 'TotalPopulation'], columns='Measure', 
                                                                     values='Data_Value').reset_index().rename_axis(None, axis=1)
#display(df_sdoh_essential_wide.head())
df_sdoh_essential_wide.shape

(32336, 11)

In [78]:
df_sdoh_essential_wide.head()

,ZIP,TotalPopulation,Crowding among housing units,Housing cost burden among households,No broadband internet subscription among households,No high school diploma among adults aged 25 years or older,Persons aged 65 years or older,Persons living below 150% of the poverty level,Persons of racial or ethnic minority status,Single-parent households,Unemployment among people 16 years and older in the labor force
0,01001,16088,0.8,24.0,14.9,7.5,22.5,11.7,13.0,5.8,5.4
1,01002,27323,2.4,38.2,7.1,2.5,13.0,30.7,29.4,5.0,7.0
2,01003,13711,0.0,0.0,33.3,0.0,0.0,8.1,38.1,0.0,18.9
3,01005,4947,0.9,15.6,8.2,3.4,13.4,13.4,4.6,5.1,4.2
4,01007,15304,0.7,19.2,5.7,2.4,16.5,10.2,10.6,3.3,2.5


In [ ]:
df_income_1901_essential, df_health_outcomes_essential_wide, df_sdoh_essential_wide

In [90]:
combined_dataset_zip=pd.merge(pd.merge(pd.merge(df_income_1901_essential, df_health_outcomes_essential_wide, on="ZIP", how="outer"), 
         df_sdoh_essential_wide, on="ZIP", how="outer"), df_acs_edited, on="ZIP", how="outer")

In [92]:
combined_dataset_zip.to_pickle(output_path+"\\combined_dataset_zip.pkl")

In [ ]:
df_state_politics_edited, df_state_public_spend_edited